In [119]:
# Update der Datensätze in der Tabelle dbo.broccolivalues -> Erweiterung mit Erntedaten
%reset
import pyodbc
import pandas as pd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import numpy as np

# CSV einlesen und Datensätze filtern, die in DB geschrieben werden sollen
#filename = ''
#cropValues = pd.read_csv(filename, delimiter=';')

# Azure-DB Verbindungsaufbau
server = 'deepbroccoliserver.database.windows.net'
database = 'DeepBroccoliDatabase'
username = 'ntb'
password = 'brokkoli_2019'
driver= '{SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

# Abfrage der Brokkoli-Daten aus dbo.broccoli
query = 'SELECT * FROM dbo.broccoli'
SQL_Query = pd.read_sql_query(query, conn)
broccoli_data = pd.DataFrame(SQL_Query)

coords = []
for index, row in broccoli_data.iterrows():
    coords.append(Point(row['lat'], row['long']))
    
broccoli_data['coord'] = coords
broccoli_data

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


,id,lat,long,cropMissing,cropOverripe,cropRudimentary,cropRotten,cropUnripe,cropWeight,cropNotice,cropNoBlossom,coord
0,1,47.217185,9.492887,True,True,True,True,True,None,None,True,POINT (47.21718464 9.492887284)
1,2,47.217187,9.492902,False,False,False,False,False,None,None,False,POINT (47.21718655 9.492901635999999)
2,3,47.217186,9.492892,False,False,False,False,False,None,None,False,POINT (47.217186 9.492892438)
3,4,47.217191,9.492904,False,False,False,False,False,None,None,False,POINT (47.21719056 9.492903630000001)
4,5,47.217190,9.492893,False,False,False,False,False,None,None,False,POINT (47.2171896 9.492892506)
5,6,47.217194,9.492895,False,False,False,False,False,None,None,False,POINT (47.21719358 9.492895280000001)
6,7,47.217193,9.492905,False,False,False,False,False,None,None,False,POINT (47.21719342 9.492905485)
7,8,47.217196,9.492908,False,False,False,False,False,None,None,False,POINT (47.21719618 9.492907670999999)
8,9,47.217197,9.492897,False,False,False,False,False,None,None,False,POINT (47.21719733 9.492897336)
9,10,47.217200,9.492899,False,False,False,False,False,None,None,False,POINT (47.21720001 9.492899415)


In [120]:
filename = 'Zusammenfügen_von_Koordinaten_Ernte.csv'
cropValues = pd.read_csv(filename, delimiter=';')

coords = []
for index, row in cropValues.iterrows():
    coords.append(Point(row['latitude'], row['longitude']))
    
cropValues['coord'] = coords
cropValues['sqlID'] = 0

In [121]:
coordsMultiPoint = MultiPoint(broccoli_data['coord'])
for index, row in cropValues.iterrows():
    coordCrop = row['coord']
    nearest_geoms = nearest_points(coordCrop, coordsMultiPoint)
    coord = nearest_geoms[1]
    broccoli = broccoli_data.loc[(broccoli_data['lat'] == coord.x) & (broccoli_data['long'] == coord.y)]
    cropValues.at[index, 'sqlID'] = broccoli['id']
    
cropValues

,id_measurement,latitude,longitude,id_ernte,fehlt,ueberreif,verkuemmert,keinKopf,faul,unreif,Gewicht,coord,sqlID
0,Hofmaenner_F_001.001,47.218537,9.493662,Hofmaenner_F_001.001,False,False,False,False,False,False,490,POINT (47.2185367 9.493662494000001),2566
1,Hofmaenner_F_001.002,47.218533,9.493660,Hofmaenner_F_001.002,False,False,False,False,False,False,520,POINT (47.21853346 9.493660373999999),2560
2,Hofmaenner_F_001.003,47.218530,9.493659,Hofmaenner_F_001.003,False,False,False,False,False,False,515,POINT (47.2185305 9.49365884),2556
3,Hofmaenner_F_001.004,47.218527,9.493657,Hofmaenner_F_001.004,False,False,False,False,False,False,562,POINT (47.21852749 9.493657413999999),2550
4,Hofmaenner_F_001.005,47.218524,9.493655,Hofmaenner_F_001.005,False,False,False,False,False,False,431,POINT (47.21852401 9.493655409),2548
5,Hofmaenner_F_001.006,47.218522,9.493654,Hofmaenner_F_001.006,False,False,False,False,False,False,790,POINT (47.21852167 9.493653867999999),2538
6,Hofmaenner_F_001.007,47.218519,9.493652,Hofmaenner_F_001.007,False,False,False,False,False,False,392,POINT (47.21851861 9.49365216),2533
7,Hofmaenner_F_001.008,47.218516,9.493651,Hofmaenner_F_001.008,False,False,False,False,False,False,491,POINT (47.2185157 9.493650618),2527
8,Hofmaenner_F_001.009,47.218512,9.493649,Hofmaenner_F_001.009,False,False,False,False,False,False,766,POINT (47.21851213 9.493648734000001),2516
9,Hofmaenner_F_001.010,47.218509,9.493647,Hofmaenner_F_001.010,False,False,False,False,False,False,575,POINT (47.21850939 9.493646902),2515


In [122]:
print(len(cropValues))
cropValues.drop_duplicates(['sqlID'], keep=False, inplace=True)
print(len(cropValues))
duplicates = cropValues[cropValues.duplicated(['sqlID'])]
print(len(duplicates))

2681
2675
0


In [123]:
cropValues.fehlt = cropValues.fehlt.astype(int)
cropValues.ueberreif = cropValues.ueberreif.astype(int)
cropValues.verkuemmert = cropValues.verkuemmert.astype(int)
cropValues.keinKopf = cropValues.keinKopf.astype(int)
cropValues.faul = cropValues.faul.astype(int)
cropValues.unreif = cropValues.unreif.astype(int)

In [126]:
 # UPDATE
#for index, row in cropValues.iterrows():
cursor = conn.cursor()
for index, row in cropValues.iterrows():    
   insertStatement = "Update dbo.broccoli set CropMissing=?, cropOverripe=?, cropRudimentary=?, cropRotten=?, cropUnripe=?, cropNoBlossom=?, cropWeight=? where dbo.broccoli.id=?"
   values = (row['fehlt'], row['ueberreif'], row['verkuemmert'], row['faul'], row['unreif'], row['keinKopf'], row['Gewicht'], row['sqlID'])     
   print(values)
   cursor.execute(insertStatement, values)

conn.commit()
conn.close()

(0, 0, 0, 0, 0, 0, 490, 2566)
(0, 0, 0, 0, 0, 0, 520, 2560)
(0, 0, 0, 0, 0, 0, 515, 2556)
(0, 0, 0, 0, 0, 0, 562, 2550)
(0, 0, 0, 0, 0, 0, 431, 2548)
(0, 0, 0, 0, 0, 0, 790, 2538)
(0, 0, 0, 0, 0, 0, 392, 2533)
(0, 0, 0, 0, 0, 0, 491, 2527)
(0, 0, 0, 0, 0, 0, 766, 2516)
(0, 0, 0, 0, 0, 0, 575, 2515)
(0, 0, 0, 0, 0, 0, 612, 2511)
(0, 0, 0, 0, 0, 0, 480, 2502)
(0, 0, 0, 0, 0, 0, 643, 2494)
(0, 0, 0, 0, 0, 0, 481, 2483)
(0, 0, 1, 0, 0, 0, 50000, 2479)
(0, 0, 0, 0, 0, 0, 473, 2476)
(0, 0, 0, 0, 0, 0, 700, 2468)
(0, 0, 0, 0, 0, 0, 512, 2459)
(0, 0, 0, 0, 0, 0, 682, 2453)
(0, 0, 0, 0, 0, 0, 466, 2448)
(0, 0, 0, 0, 0, 0, 600, 2442)
(0, 0, 0, 0, 0, 0, 549, 2435)
(0, 0, 0, 0, 0, 0, 540, 2430)
(0, 0, 0, 0, 0, 0, 622, 2425)
(0, 0, 0, 0, 0, 0, 453, 2417)
(0, 0, 0, 0, 0, 0, 729, 2413)
(0, 0, 0, 0, 0, 0, 458, 2406)
(0, 0, 0, 0, 0, 0, 422, 2402)
(0, 0, 0, 0, 0, 0, 573, 2394)
(0, 0, 0, 0, 0, 0, 480, 2389)
(0, 0, 0, 0, 0, 0, 667, 2382)
(0, 0, 0, 0, 0, 0, 642, 2375)
(0, 0, 0, 0, 0, 0, 315, 2372)
(0, 0, 0

(0, 0, 0, 0, 0, 0, 627, 930)
(0, 0, 0, 0, 0, 0, 534, 920)
(0, 0, 0, 0, 0, 0, 577, 919)
(0, 0, 0, 0, 0, 0, 603, 909)
(0, 0, 0, 0, 0, 0, 703, 901)
(0, 0, 0, 0, 0, 0, 571, 898)
(0, 0, 0, 0, 0, 0, 726, 891)
(0, 0, 0, 0, 0, 0, 519, 884)
(0, 0, 0, 0, 0, 0, 672, 877)
(0, 0, 0, 0, 0, 0, 730, 873)
(0, 0, 0, 0, 0, 0, 553, 866)
(0, 0, 0, 0, 0, 0, 825, 861)
(0, 0, 0, 0, 0, 0, 680, 855)
(0, 0, 0, 0, 0, 0, 693, 849)
(0, 0, 1, 0, 0, 0, 50000, 844)
(0, 0, 0, 0, 0, 0, 762, 841)
(0, 0, 0, 0, 0, 0, 724, 831)
(0, 0, 0, 0, 0, 0, 554, 826)
(0, 0, 0, 0, 0, 0, 522, 820)
(0, 0, 0, 0, 0, 0, 751, 819)
(0, 0, 0, 0, 0, 0, 405, 811)
(0, 0, 0, 0, 1, 0, 50000, 808)
(0, 0, 0, 0, 0, 0, 639, 801)
(0, 0, 0, 0, 0, 0, 674, 795)
(0, 0, 0, 0, 0, 0, 478, 785)
(0, 0, 0, 0, 0, 0, 745, 779)
(0, 0, 0, 0, 0, 0, 597, 777)
(0, 0, 0, 0, 0, 0, 711, 765)
(0, 1, 0, 0, 0, 0, 50000, 760)
(0, 0, 0, 0, 0, 0, 384, 754)
(0, 0, 0, 0, 0, 0, 614, 750)
(0, 0, 0, 0, 0, 0, 574, 743)
(0, 0, 0, 0, 0, 0, 728, 738)
(0, 0, 0, 0, 0, 0, 327, 731)
(0, 0, 0

(0, 0, 0, 0, 0, 0, 50000, 1877)
(0, 0, 0, 0, 0, 0, 50000, 1876)
(0, 0, 0, 0, 0, 0, 50000, 1871)
(0, 0, 0, 0, 0, 0, 527, 1860)
(0, 0, 0, 0, 0, 0, 836, 1855)
(0, 0, 0, 0, 0, 0, 553, 1847)
(0, 0, 0, 0, 0, 0, 584, 1841)
(0, 0, 0, 0, 0, 0, 627, 1835)
(0, 0, 0, 0, 0, 0, 630, 1834)
(0, 0, 0, 0, 0, 0, 649, 1824)
(0, 0, 0, 0, 0, 0, 625, 1815)
(0, 0, 0, 0, 0, 0, 378, 1814)
(0, 0, 0, 0, 0, 0, 586, 1810)
(0, 0, 0, 0, 0, 0, 521, 1797)
(0, 0, 0, 0, 0, 0, 633, 1791)
(0, 0, 0, 0, 0, 0, 502, 1785)
(0, 0, 0, 0, 0, 0, 583, 1780)
(0, 0, 0, 0, 0, 0, 672, 1772)
(0, 0, 0, 0, 1, 0, 168, 1771)
(0, 0, 0, 0, 0, 0, 693, 1762)
(0, 0, 0, 0, 0, 0, 500, 1754)
(0, 0, 0, 0, 0, 0, 463, 1749)
(0, 0, 0, 0, 0, 0, 605, 1743)
(0, 0, 0, 0, 0, 0, 558, 1737)
(0, 0, 0, 0, 0, 0, 560, 1731)
(0, 0, 0, 0, 0, 0, 742, 1724)
(0, 0, 0, 0, 0, 0, 307, 1723)
(0, 0, 0, 0, 0, 0, 524, 1716)
(0, 0, 0, 0, 0, 0, 590, 1712)
(0, 0, 0, 0, 0, 0, 429, 1702)
(0, 0, 0, 0, 0, 0, 666, 1698)
(0, 0, 0, 0, 0, 0, 416, 1696)
(0, 0, 0, 0, 0, 0, 518, 1684)
(0, 

(0, 0, 0, 0, 0, 0, 745, 239)
(0, 0, 0, 0, 1, 0, 215, 237)
(0, 0, 0, 0, 0, 0, 641, 227)
(0, 0, 0, 0, 0, 0, 431, 219)
(0, 0, 0, 0, 0, 0, 440, 215)
(0, 0, 0, 0, 0, 0, 319, 208)
(0, 0, 0, 0, 0, 0, 682, 202)
(0, 0, 0, 0, 0, 0, 590, 197)
(0, 0, 0, 0, 0, 0, 509, 191)
(0, 0, 0, 0, 0, 0, 492, 184)
(0, 0, 0, 0, 0, 0, 648, 177)
(0, 0, 0, 0, 0, 0, 616, 172)
(0, 0, 0, 0, 0, 0, 598, 166)
(0, 0, 0, 0, 1, 0, 291, 159)
(0, 0, 0, 0, 0, 0, 564, 152)
(0, 0, 0, 0, 0, 0, 485, 151)
(0, 0, 0, 0, 0, 0, 463, 140)
(0, 0, 0, 0, 0, 0, 723, 135)
(0, 0, 0, 0, 1, 0, 259, 128)
(0, 0, 0, 0, 0, 0, 559, 121)
(0, 0, 0, 0, 0, 0, 570, 116)
(0, 0, 0, 0, 0, 0, 607, 111)
(0, 0, 0, 0, 0, 0, 665, 105)
(0, 0, 0, 0, 0, 0, 612, 99)
(0, 0, 0, 0, 0, 0, 614, 92)
(0, 0, 0, 0, 0, 0, 636, 85)
(0, 0, 0, 0, 0, 0, 452, 78)
(0, 0, 0, 0, 0, 0, 712, 74)
(0, 0, 0, 0, 0, 0, 440, 71)
(0, 0, 0, 0, 0, 0, 403, 66)
(0, 0, 0, 0, 0, 0, 322, 62)
(0, 0, 0, 0, 0, 0, 439, 56)
(0, 0, 0, 0, 0, 0, 491, 54)
(0, 0, 0, 0, 1, 0, 290, 49)
(0, 1, 0, 0, 0, 0, 50000,

(0, 0, 0, 0, 0, 0, 666, 1368)
(0, 0, 0, 0, 0, 0, 643, 1366)
(0, 0, 0, 0, 0, 0, 617, 1360)
(0, 0, 0, 0, 0, 0, 900, 1355)
(0, 0, 0, 0, 0, 0, 339, 1349)
(0, 0, 0, 0, 0, 0, 588, 1343)
(0, 0, 0, 0, 0, 0, 768, 1338)
(0, 0, 0, 0, 0, 0, 424, 1329)
(0, 0, 0, 0, 0, 0, 745, 1326)
(0, 0, 0, 0, 0, 0, 424, 1320)
(0, 0, 0, 0, 0, 0, 871, 1315)
(0, 0, 0, 0, 0, 0, 667, 1309)
(0, 0, 0, 0, 1, 0, 298, 1303)
(0, 0, 0, 0, 0, 0, 646, 1297)
(0, 0, 0, 0, 0, 0, 515, 1290)
(0, 0, 0, 0, 0, 0, 405, 1284)
(0, 0, 0, 0, 0, 0, 836, 1281)
(0, 0, 0, 0, 0, 0, 593, 1273)
(0, 0, 0, 0, 0, 0, 786, 1264)
(0, 0, 0, 0, 0, 0, 413, 1260)
(0, 0, 0, 0, 0, 0, 694, 1256)
(0, 0, 0, 0, 0, 0, 936, 1248)
(0, 0, 1, 0, 0, 0, 50000, 1237)
(0, 0, 0, 0, 0, 0, 851, 1230)
(0, 0, 0, 0, 0, 0, 642, 1229)
(0, 0, 0, 0, 0, 0, 520, 1219)
(0, 0, 0, 0, 0, 0, 683, 1218)
(0, 0, 0, 0, 0, 0, 616, 1212)
(0, 0, 0, 0, 0, 0, 785, 1205)
(0, 0, 1, 0, 0, 0, 50000, 1202)
(0, 0, 0, 0, 0, 0, 757, 1194)
(0, 0, 0, 0, 0, 0, 441, 1191)
(0, 0, 0, 0, 0, 0, 741, 1185)
(0, 0,

(0, 0, 0, 0, 0, 0, 342, 2423)
(0, 0, 0, 0, 0, 0, 340, 2416)
(0, 0, 0, 0, 0, 0, 520, 2411)
(0, 0, 0, 0, 0, 0, 756, 2410)
(0, 0, 0, 0, 0, 0, 354, 2399)
(0, 0, 0, 0, 0, 0, 516, 2398)
(0, 0, 0, 0, 0, 0, 630, 2387)
(0, 0, 0, 0, 0, 0, 730, 2381)
(0, 0, 0, 0, 0, 0, 751, 2380)
(0, 0, 1, 0, 0, 0, 50000, 2371)
(0, 0, 0, 0, 0, 0, 560, 2368)
(0, 0, 0, 0, 0, 0, 688, 2358)
(0, 0, 0, 0, 0, 0, 474, 2354)
(0, 0, 0, 0, 0, 0, 656, 2346)
(0, 0, 0, 0, 0, 0, 312, 2343)
(0, 0, 0, 0, 0, 0, 527, 2335)
(0, 0, 0, 0, 1, 0, 287, 2329)
(0, 0, 0, 0, 0, 0, 342, 2322)
(0, 0, 0, 0, 0, 0, 540, 2316)
(0, 0, 0, 0, 0, 0, 446, 2311)
(0, 0, 0, 0, 0, 0, 444, 2305)
(0, 0, 0, 0, 0, 0, 621, 2298)
(0, 0, 0, 0, 0, 0, 477, 2291)
(0, 0, 0, 0, 0, 0, 345, 2286)
(0, 0, 0, 0, 0, 0, 385, 2280)
(0, 0, 0, 0, 0, 0, 413, 2274)
(0, 0, 0, 0, 0, 0, 446, 2269)
(0, 0, 0, 0, 0, 0, 501, 2262)
(0, 0, 0, 0, 0, 0, 502, 2256)
(0, 0, 0, 0, 0, 0, 375, 2255)
(0, 0, 0, 0, 0, 0, 435, 2245)
(0, 0, 0, 0, 0, 0, 439, 2238)
(0, 0, 0, 0, 0, 0, 447, 2233)
(0, 0, 0

(0, 0, 0, 0, 0, 0, 651, 827)
(0, 0, 0, 0, 0, 0, 371, 822)
(0, 0, 0, 0, 0, 0, 383, 816)
(0, 0, 0, 0, 0, 0, 504, 813)
(0, 0, 0, 0, 0, 0, 360, 807)
(0, 0, 0, 0, 0, 0, 488, 799)
(0, 0, 0, 0, 0, 0, 538, 792)
(0, 0, 0, 0, 0, 0, 515, 788)
(0, 0, 0, 0, 1, 0, 118, 783)
(0, 0, 0, 0, 0, 0, 588, 775)
(0, 0, 0, 0, 0, 0, 589, 771)
(0, 0, 0, 0, 0, 0, 521, 764)
(0, 0, 0, 0, 0, 0, 416, 758)
(0, 0, 0, 0, 0, 0, 410, 753)
(0, 0, 0, 0, 0, 0, 509, 747)
(0, 0, 0, 0, 0, 0, 374, 741)
(0, 0, 0, 0, 0, 0, 603, 739)
(0, 0, 1, 0, 0, 0, 50000, 729)
(0, 0, 0, 0, 0, 0, 635, 723)
(0, 0, 0, 0, 0, 0, 422, 716)
(0, 0, 0, 0, 0, 0, 473, 712)
(0, 0, 0, 0, 0, 0, 499, 705)
(0, 0, 0, 0, 0, 0, 417, 699)
(0, 0, 0, 0, 0, 0, 542, 693)
(0, 0, 0, 0, 0, 0, 456, 687)
(0, 0, 0, 0, 0, 0, 446, 682)
(0, 0, 0, 0, 0, 0, 316, 677)
(0, 0, 0, 0, 0, 0, 614, 669)
(0, 0, 0, 0, 0, 0, 323, 665)
(0, 0, 0, 0, 0, 0, 403, 663)
(0, 0, 0, 0, 0, 0, 411, 650)
(0, 0, 0, 0, 0, 0, 508, 645)
(0, 0, 0, 0, 0, 0, 573, 640)
(0, 0, 0, 0, 0, 0, 387, 635)
(0, 0, 0, 0,

(0, 0, 0, 0, 0, 0, 555, 1952)
(0, 0, 0, 0, 0, 0, 689, 1946)
(0, 0, 0, 0, 0, 0, 546, 1939)
(0, 0, 0, 0, 0, 0, 502, 1932)
(0, 0, 0, 0, 0, 0, 520, 1926)
(0, 0, 0, 0, 0, 0, 452, 1921)
(0, 0, 0, 0, 0, 0, 730, 1917)
(0, 0, 0, 0, 0, 0, 612, 1911)
(0, 0, 0, 0, 0, 0, 512, 1905)
(0, 0, 0, 0, 0, 0, 648, 1899)
(0, 0, 0, 0, 0, 0, 514, 1891)
(0, 0, 0, 0, 1, 0, 246, 1887)
(0, 0, 0, 0, 0, 0, 650, 1880)
(0, 0, 0, 0, 0, 0, 491, 1872)
(0, 0, 0, 0, 0, 0, 667, 1869)
(0, 0, 0, 0, 0, 0, 532, 1862)
(0, 0, 0, 0, 0, 0, 730, 1853)
(0, 0, 0, 0, 1, 0, 50000, 1851)
(0, 0, 0, 0, 0, 0, 442, 1844)
(0, 0, 0, 0, 0, 0, 509, 1839)
(0, 0, 0, 0, 0, 0, 600, 1831)
(0, 0, 0, 0, 0, 0, 588, 1826)
(0, 0, 0, 0, 0, 0, 582, 1821)
(0, 0, 0, 0, 0, 0, 705, 1816)
(0, 0, 0, 0, 1, 0, 50000, 1809)
(0, 0, 0, 0, 0, 0, 811, 1803)
(0, 0, 0, 0, 0, 0, 573, 1796)
(0, 0, 0, 0, 0, 0, 433, 1792)
(0, 0, 0, 0, 0, 0, 588, 1786)
(0, 0, 0, 0, 1, 0, 50000, 1778)
(0, 0, 0, 0, 0, 0, 700, 1775)
(0, 0, 0, 0, 0, 0, 495, 1766)
(0, 0, 0, 0, 0, 0, 554, 1760)
(0, 

(0, 0, 0, 0, 0, 0, 350, 214)
(0, 0, 0, 0, 0, 0, 363, 201)
(0, 0, 0, 0, 0, 0, 455, 196)
(0, 0, 0, 0, 0, 0, 455, 194)
(0, 0, 0, 0, 0, 0, 459, 185)
(0, 0, 0, 0, 0, 0, 338, 183)
(0, 0, 0, 0, 0, 0, 368, 173)
(0, 0, 0, 0, 0, 0, 545, 169)
(0, 0, 0, 0, 1, 0, 274, 165)
(0, 0, 0, 0, 0, 0, 517, 156)
(0, 0, 0, 0, 0, 0, 357, 154)
(0, 0, 0, 1, 0, 0, 50000, 141)
(0, 0, 0, 0, 0, 0, 415, 138)
(0, 0, 0, 0, 0, 0, 345, 131)
(0, 0, 1, 0, 0, 0, 50000, 125)
(0, 0, 1, 0, 0, 0, 50000, 120)
(0, 0, 1, 0, 0, 0, 50000, 115)
(0, 0, 1, 0, 0, 0, 50000, 107)
(0, 0, 1, 0, 0, 0, 50000, 104)
(0, 0, 1, 0, 0, 0, 50000, 95)
(0, 0, 0, 0, 1, 0, 133, 91)
(0, 0, 0, 0, 1, 0, 188, 87)
(0, 0, 0, 0, 1, 0, 120, 82)
(0, 0, 0, 0, 0, 0, 418, 2681)
(0, 0, 0, 0, 0, 0, 904, 2678)
(0, 0, 1, 0, 0, 0, 50000, 2677)
(0, 0, 0, 0, 0, 0, 445, 2675)
(0, 0, 0, 0, 0, 0, 730, 2672)
(0, 0, 0, 0, 0, 0, 517, 2670)
(0, 0, 0, 0, 0, 0, 665, 2668)
(0, 0, 0, 0, 0, 0, 491, 2666)
(0, 0, 0, 0, 0, 0, 448, 2664)
(0, 0, 0, 0, 0, 0, 594, 2663)
(0, 0, 0, 0, 1, 0, 27

(0, 0, 0, 0, 0, 0, 681, 1206)
(0, 0, 0, 0, 0, 0, 462, 1197)
(0, 0, 0, 0, 0, 0, 611, 1196)
(0, 0, 0, 0, 0, 0, 616, 1186)
(0, 0, 1, 0, 0, 0, 50000, 1179)
(0, 0, 0, 0, 0, 0, 1042, 1173)
(0, 0, 0, 0, 0, 0, 655, 1169)
(0, 0, 0, 0, 0, 0, 721, 1162)
(0, 0, 0, 0, 0, 0, 655, 1144)
(0, 0, 0, 0, 0, 0, 410, 1142)
(0, 0, 0, 0, 1, 0, 83, 1135)
(0, 0, 0, 0, 0, 0, 628, 1134)
(0, 0, 0, 0, 0, 0, 483, 1128)
(0, 0, 0, 0, 0, 0, 667, 1121)
(0, 0, 0, 0, 0, 0, 750, 1110)
(0, 0, 0, 0, 0, 0, 395, 1104)
(0, 0, 0, 0, 0, 0, 600, 1095)
(0, 0, 0, 0, 0, 0, 685, 1089)
(0, 0, 0, 0, 0, 0, 785, 1083)
(0, 0, 0, 0, 0, 0, 632, 1082)
(0, 0, 0, 0, 0, 0, 337, 1071)
(0, 0, 0, 0, 0, 0, 611, 1065)
(0, 0, 0, 0, 0, 0, 652, 1064)
(0, 0, 0, 0, 0, 0, 444, 1053)
(0, 0, 0, 0, 0, 0, 821, 1047)
(0, 0, 0, 0, 0, 0, 791, 1043)
(0, 0, 0, 0, 0, 0, 482, 1032)
(0, 0, 0, 0, 0, 0, 453, 1029)
(0, 0, 0, 0, 0, 0, 668, 1019)
(0, 0, 0, 0, 0, 0, 440, 1017)
(0, 0, 0, 0, 0, 0, 653, 1011)
(0, 0, 0, 0, 0, 0, 760, 1001)
(0, 0, 0, 0, 0, 0, 959, 990)
(0, 0, 0,